# Architecture Reconstruction for Zeeguu

This notebook provides a comprehensive architecture reconstruction for the Zeeguu system, analyzing both the backend API (`../Data/api`) and frontend (`../Data/frontend`). The goal is to extract meaningful architectural views that provide insights into the system's structure, dependencies, and complexity.

## 1. Setup and Installation

Install all the necessary packages for our analysis.

In [ ]:
import sys
import subprocess

def install_package(package):
    package_name = package.split('==')[0] if '==' in package else package
    try:
        __import__(package_name)
        print(f"{package_name} is already installed.")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
install_package('prospector')
install_package('pylint')
install_package('networkx')
install_package('matplotlib')
install_package('pyyaml')
install_package('jinja2')
install_package('pandas')
install_package('graphviz')
install_package('pydot')

prospector is already installed.
pylint is already installed.
networkx is already installed.
matplotlib is already installed.
Installing pyyaml...
jinja2 is already installed.
pandas is already installed.
graphviz is already installed.
pydot is already installed.


## 1. Setup, & Basic System Information & basic prospector


In [ ]:
import os
import subprocess
import json
import networkx as nx

# Define absolute paths based on your project structure
API_PATH = r"C:\Programming\Architecture\ArchitecturalRecovery\Data\api"
FRONTEND_PATH = r"C:\Programming\Architecture\ArchitecturalRecovery\Data\web"
OUTPUT_DIR = r"C:\Programming\Architecture\ArchitecturalRecovery\output\prospector"  # Adjust as needed

def count_files_and_lines(path, extensions=None):
    """
    Count files and lines of code in the given path, filtered by file extensions.
    """
    if extensions is None:
        extensions = ['.py', '.js', '.jsx', '.ts', '.tsx']
    
    file_count = 0
    line_count = 0
    files_by_ext = {ext: 0 for ext in extensions}
    lines_by_ext = {ext: 0 for ext in extensions}
    
    for root, _, files in os.walk(path):
        for file in files:
            _, ext = os.path.splitext(file)
            if ext in extensions:
                file_count += 1
                files_by_ext[ext] += 1
                
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        lines = len(f.readlines())
                        line_count += lines
                        lines_by_ext[ext] += lines
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
    
    return {
        'total_files': file_count,
        'total_lines': line_count,
        'files_by_ext': files_by_ext,
        'lines_by_ext': lines_by_ext
    }

# Analyze backend
backend_stats = count_files_and_lines(API_PATH)
print("Backend Statistics:")
print(json.dumps(backend_stats, indent=2))

def run_prospector(path, output_file):
    """
    Run Prospector on the specified path and save results to output_file.
    Returns a summary of findings.
    """
    full_output_path = os.path.join(OUTPUT_DIR, output_file)
    
    try:
        # Run Prospector without forcing a check on the exit status
        result = subprocess.run(
            ['python', '-m', 'prospector', path, f'--output-format=json:{output_file}'],
            capture_output=True, text=True, check=False
        )
        
        # If there's any stderr output, print it for debugging purposes
        if result.stderr:
            print(f"Prospector stderr for {path}:")
            print(result.stderr)
        
        # Attempt to parse JSON output even if exit status is non-zero
        try:
            results = json.loads(result.stdout)
        except Exception as parse_error:
            print(f"Error parsing Prospector JSON output for {path}: {parse_error}")
            return None

        # Write the output to a file
        with open(full_output_path, 'w', encoding='utf-8') as f:
            f.write(result.stdout)
        
        print(f"Prospector analysis completed for {path}")
        
        messages = results.get('messages', [])
        tools_used = results.get('tools', [])
        summary = {
            'total_issues': len(messages),
            'tools_used': tools_used,
            'issues_by_type': {}
        }
        
        for msg in messages:
            msg_type = msg.get('source')
            if msg_type not in summary['issues_by_type']:
                summary['issues_by_type'][msg_type] = 0
            summary['issues_by_type'][msg_type] += 1
            
        return summary
    except Exception as e:
        print(f"Error running Prospector on {path}: {e}")
        return None

# Run Prospector on backend
backend_prospector = run_prospector(API_PATH, 'api_prospector_report.json')
print("\nBackend Prospector Summary:")
print(json.dumps(backend_prospector, indent=2))


Backend Statistics:
{
  "total_files": 357,
  "total_lines": 31256,
  "files_by_ext": {
    ".py": 357,
    ".js": 0,
    ".jsx": 0,
    ".ts": 0,
    ".tsx": 0
  },
  "lines_by_ext": {
    ".py": 31256,
    ".js": 0,
    ".jsx": 0,
    ".ts": 0,
    ".tsx": 0
  }
}
Error parsing Prospector JSON output for C:\Programming\Architecture\ArchitecturalRecovery\Data\api: Expecting value: line 1 column 1 (char 0)

Backend Prospector Summary:
null
